In [24]:
import os
import keras as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [2]:
file = 'dataset_1'

train_datagen = K.preprocessing.image.ImageDataGenerator()

validation_datagen = K.preprocessing.image.ImageDataGenerator()

test_datagen = K.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        os.path.join(file, 'train'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical',
        )

validation_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'validation'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

test_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'test'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

Found 12556 images belonging to 28 classes.
Found 1710 images belonging to 28 classes.
Found 3515 images belonging to 28 classes.


In [3]:
def runGenerator(generator):
    X = []
    y = []
    for i in range(len(generator)):
        X_ex, y_ex = generator.next()
        X.append(X_ex)
        y.append(y_ex)
    return X, y

def F1ScoreCalculator(test_generatorm, model):
    X_test = []
    y_test = []
    y_pred = []
    for i in range(len(test_generator)):
        X_ex, y_ex = test_generator.next()
        X_test.append(X_ex)
        y_test.append(y_ex.argmax(axis=-1))
        y_pred.append(model.predict(X_ex).argmax(axis=-1))

    return f1_score(y_test, y_pred, average='micro')

In [13]:
def Create_model(kernel_sizes=5, neurons_bk1=20, neurons_bk2=50, learn_rate=0.001, optomizer='sgd', activation='relu'):
    print('kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
        kernel_sizes, 
        neurons_bk1, 
        neurons_bk2, 
        learn_rate,
        optomizer,
        activation))
    
    model = Sequential()

    model.add(Conv2D(neurons_bk1, kernel_size=kernel_sizes, padding="same", input_shape=(50, 50, 1)))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(neurons_bk2, kernel_size=kernel_sizes, border_mode="same"))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(500))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())

    model.add(Dense(28))
    model.add(Activation("softmax"))
    
#     model.summary()
    if (optomizer == 'sgd'):
        model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=learn_rate), metrics=["accuracy"])
    else:
        model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learn_rate), metrics=["accuracy"])
    return model

In [6]:
early_stop = K.callbacks.EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='max')

In [10]:

kernel_sizes=[3, 5]
neurons_bk1=[10, 20, 30]
neurons_bk2=[50, 100]
learn_rate=[0.01, 0.005]
optomizer=['sgd', 'adam']
activation=['relu', 'leakrelu']



In [11]:
INPUT_SHAPE = (50, 50, 1)
NB_EPOCH = 20
BATCH_SIZE = 32
VERBOSE = 1
OPTIMIZER = SGD()
CLASSES = 28
STEPS = (int)(train_generator.n/BATCH_SIZE)

In [29]:
max_f1 = 0
for ks in kernel_sizes:
    for n_bk1 in neurons_bk1:
        for n_bk2 in  neurons_bk2:
            for lr in learn_rate:
                for opt in optomizer:
                    for act in activation:
                        model = Create_model(kernel_sizes=ks,
                                            neurons_bk1=n_bk1,
                                            neurons_bk2=n_bk2,
                                            learn_rate=lr,
                                            optomizer=opt,
                                            activation=act
                                            )
                        history = model.fit_generator(
                            train_generator,
                            steps_per_epoch=STEPS,
                            epochs=NB_EPOCH,
                            verbose=VERBOSE,
                            validation_data=validation_generator,
                    #         validation_steps=STEPS,
                            callbacks = [early_stop]
                        )
                        f1_score_val = F1ScoreCalculator(test_generator, model)
                        print ("F1-Score = {}".format(f1_score_val))
                        if (f1_score_val > max_f1):
                            best_model = model
                            max_f1 = f1_score_val
                            printer = 'kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
                                    ks, 
                                    n_bk1, 
                                    n_bk2, 
                                    lr,
                                    opt,
                                    act)
                            print ('Best F1-Score')
                            
                        del model
                        del history
                        K.backend.clear_session()
                        K.backend.get_session()
                
                        print ('------ END MODEL ------\n\n')

print(printer)

kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=relu


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=3, padding="same")`


Epoch 1/20
392/392 [==============================] - 6s 16ms/step - loss: 15.4602 - acc: 0.0408 - val_loss: 15.5420 - val_acc: 0.0351
Epoch 2/20
392/392 [==============================] - 6s 15ms/step - loss: 15.5836 - acc: 0.0332 - val_loss: 15.5420 - val_acc: 0.0351
Epoch 3/20
392/392 [==============================] - 6s 15ms/step - loss: 15.4393 - acc: 0.0408 - val_loss: 15.5243 - val_acc: 0.0368
Epoch 4/20
392/392 [==============================] - 6s 15ms/step - loss: 15.4191 - acc: 0.0434 - val_loss: 15.5243 - val_acc: 0.0368
Epoch 5/20
392/392 [==============================] - 6s 15ms/step - loss: 15.6658 - acc: 0.0281 - val_loss: 15.5243 - val_acc: 0.0368
Epoch 6/20
392/392 [==============================] - 6s 15ms/step - loss: 15.7480 - acc: 0.0230 - val_loss: 15.5243 - val_acc: 0.0368
F1-Score = 0.0352773826458037
Best F1-Score
------ END MODEL ------


kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
3

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, kernel_size=3, padding="same")`


Epoch 1/20
392/392 [==============================] - 8s 22ms/step - loss: 15.6623 - acc: 0.0281 - val_loss: 15.3358 - val_acc: 0.0485
Epoch 2/20
392/392 [==============================] - 8s 20ms/step - loss: 15.7069 - acc: 0.0255 - val_loss: 15.3358 - val_acc: 0.0485
Epoch 3/20
392/392 [==============================] - 8s 20ms/step - loss: 15.5424 - acc: 0.0357 - val_loss: 15.3358 - val_acc: 0.0485
Epoch 4/20
392/392 [==============================] - 8s 20ms/step - loss: 15.7069 - acc: 0.0255 - val_loss: 15.3358 - val_acc: 0.0485
F1-Score = 0.032716927453769556
------ END MODEL ------


kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 100 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.5676 - acc: 0.0332 - val_loss: 15.6185 - val_acc: 0.0310
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.3369 - acc: 0.0485 - val_loss: 15.6185 - val_acc: 0.0310
Epoch 3/20
392/392 [===

...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, kernel_size=5, padding="same")`


Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.7516 - acc: 0.0204 - val_loss: 15.3358 - val_acc: 0.0485
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.7069 - acc: 0.0255 - val_loss: 15.3358 - val_acc: 0.0485
Epoch 3/20
392/392 [==============================] - 8s 21ms/step - loss: 15.2957 - acc: 0.0510 - val_loss: 15.3358 - val_acc: 0.0485
Epoch 4/20
392/392 [==============================] - 8s 21ms/step - loss: 15.5836 - acc: 0.0332 - val_loss: 15.3358 - val_acc: 0.0485
F1-Score = 0.032716927453769556
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 10 | neurons_bk2= 100 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.6121 - acc: 0.0306 - val_loss: 15.3640 - val_acc: 0.0468
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.6247 - acc: 0.0306 - val_loss: 15.3640 - val_acc: 0.0468
Epoch 3/20
392/392 [===

392/392 [==============================] - 8s 20ms/step - loss: 15.2546 - acc: 0.0536 - val_loss: 15.6091 - val_acc: 0.0316
F1-Score = 0.032147937411095305
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 20 | neurons_bk2= 50 | learn_rate=0.005 | optomizer=sgd | activation=relu
Epoch 1/20
392/392 [==============================] - 6s 16ms/step - loss: 15.6992 - acc: 0.0255 - val_loss: 15.5997 - val_acc: 0.0322
Epoch 2/20
392/392 [==============================] - 6s 15ms/step - loss: 15.5424 - acc: 0.0357 - val_loss: 15.5997 - val_acc: 0.0322
Epoch 3/20
392/392 [==============================] - 6s 15ms/step - loss: 15.5013 - acc: 0.0383 - val_loss: 15.5997 - val_acc: 0.0322
Epoch 4/20
392/392 [==============================] - 6s 15ms/step - loss: 15.6658 - acc: 0.0281 - val_loss: 15.5997 - val_acc: 0.0322
F1-Score = 0.03556187766714083
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 20 | neurons_bk2= 50 | learn_rate=0.005 | optomizer=sgd | activation=leakrelu
Epoch 1

392/392 [==============================] - 11s 29ms/step - loss: 15.6658 - acc: 0.0281 - val_loss: 15.6185 - val_acc: 0.0310
Epoch 3/20
392/392 [==============================] - 11s 29ms/step - loss: 15.2957 - acc: 0.0510 - val_loss: 15.6185 - val_acc: 0.0310
Epoch 4/20
392/392 [==============================] - 11s 29ms/step - loss: 15.7892 - acc: 0.0204 - val_loss: 15.6185 - val_acc: 0.0310
F1-Score = 0.032147937411095305
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 30 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=relu
Epoch 1/20
392/392 [==============================] - 6s 16ms/step - loss: 15.4743 - acc: 0.0383 - val_loss: 15.5337 - val_acc: 0.0363
Epoch 2/20
392/392 [==============================] - 6s 15ms/step - loss: 15.6247 - acc: 0.0306 - val_loss: 15.5337 - val_acc: 0.0363
Epoch 3/20
392/392 [==============================] - 6s 15ms/step - loss: 15.2957 - acc: 0.0510 - val_loss: 15.5337 - val_acc: 0.0363
Epoch 4/20
392/392 [================

Epoch 1/20
392/392 [==============================] - 12s 31ms/step - loss: 15.6081 - acc: 0.0306 - val_loss: 15.5714 - val_acc: 0.0339
Epoch 2/20
392/392 [==============================] - 11s 29ms/step - loss: 15.9125 - acc: 0.0128 - val_loss: 15.5714 - val_acc: 0.0339
Epoch 3/20
392/392 [==============================] - 12s 29ms/step - loss: 15.5836 - acc: 0.0332 - val_loss: 15.5714 - val_acc: 0.0339
Epoch 4/20
392/392 [==============================] - 11s 29ms/step - loss: 15.7892 - acc: 0.0204 - val_loss: 15.5714 - val_acc: 0.0339
F1-Score = 0.027880512091038402
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 30 | neurons_bk2= 100 | learn_rate=0.005 | optomizer=sgd | activation=relu
Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.7592 - acc: 0.0204 - val_loss: 15.5243 - val_acc: 0.0368
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.5424 - acc: 0.0357 - val_loss: 15.5243 - val_acc: 0.0368
Epoch 3/20
392/392 [==

In [ ]:
max_f1 = 0
for ks in kernel_sizes:
    for n_bk1 in neurons_bk1:
        for n_bk2 in  neurons_bk2:
            for lr in learn_rate:
#                 for opt in optomizer:
#                     for act in activation:
                        model = Create_model(kernel_sizes=ks,
                                            neurons_bk1=n_bk1,
                                            neurons_bk2=n_bk2,
                                            learn_rate=lr,
                                            optomizer='adam',
#                                             activation=act
                                            )
                        history = model.fit_generator(
                            train_generator,
                            steps_per_epoch=STEPS,
                            epochs=NB_EPOCH,
                            verbose=VERBOSE,
                            validation_data=validation_generator,
                    #         validation_steps=STEPS,
                            callbacks = [early_stop]
                        )
                        f1_score_val = F1ScoreCalculator(test_generator, model)
                        print ("F1-Score = {}".format(f1_score_val))
                        if (f1_score_val > max_f1):
                            best_model = model
                            max_f1 = f1_score_val
                            printer = 'kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
                                    ks, 
                                    n_bk1, 
                                    n_bk2, 
                                    lr,
                                    'adam',
                                    'relu')
                            print ('Best F1-Score')
                            
                        del model
                        del history
                
                        print ('------ END MODEL ------\n\n')

print(printer)


In [ ]:
max_f1 = 0
for ks in kernel_sizes:
    for n_bk1 in neurons_bk1:
        for n_bk2 in  neurons_bk2:
            for lr in learn_rate:
#                 for opt in optomizer:
#                     for act in activation:
                        model = Create_model(kernel_sizes=ks,
                                            neurons_bk1=n_bk1,
                                            neurons_bk2=n_bk2,
                                            learn_rate=lr,
#                                             optomizer='adam',
                                            activation='leakrelu'
                                            )
                        history = model.fit_generator(
                            train_generator,
                            steps_per_epoch=STEPS,
                            epochs=NB_EPOCH,
                            verbose=VERBOSE,
                            validation_data=validation_generator,
                    #         validation_steps=STEPS,
                            callbacks = [early_stop]
                        )
                        f1_score_val = F1ScoreCalculator(test_generator, model)
                        print ("F1-Score = {}".format(f1_score_val))
                        if (f1_score_val > max_f1):
                            best_model = model
                            max_f1 = f1_score_val
                            printer = 'kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
                                    ks, 
                                    n_bk1, 
                                    n_bk2, 
                                    lr,
                                    'sgd',
                                    'leakrelu')
                            print ('Best F1-Score')
                            
                        del model
                        del history
                
                        print ('------ END MODEL ------\n\n')

print(printer)


In [ ]:
max_f1 = 0
for ks in kernel_sizes:
    for n_bk1 in neurons_bk1:
        for n_bk2 in  neurons_bk2:
            for lr in learn_rate:
#                 for opt in optomizer:
#                     for act in activation:
                        model = Create_model(kernel_sizes=ks,
                                            neurons_bk1=n_bk1,
                                            neurons_bk2=n_bk2,
                                            learn_rate=lr,
                                            optomizer='adam',
                                            activation='leakrelu'
                                            )
                        history = model.fit_generator(
                            train_generator,
                            steps_per_epoch=STEPS,
                            epochs=NB_EPOCH,
                            verbose=VERBOSE,
                            validation_data=validation_generator,
                    #         validation_steps=STEPS,
                            callbacks = [early_stop]
                        )
                        f1_score_val = F1ScoreCalculator(test_generator, model)
                        print ("F1-Score = {}".format(f1_score_val))
                        if (f1_score_val > max_f1):
                            best_model = model
                            max_f1 = f1_score_val
                            printer = 'kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
                                    ks, 
                                    n_bk1, 
                                    n_bk2, 
                                    lr,
                                    'adam',
                                    'leakrelu')
                            print ('Best F1-Score')
                            
                        del model
                        del history
                
                        print ('------ END MODEL ------\n\n')

print(printer)
